# Generation of MGXS for a fuel pin cell, using OpenMC

Author: J. Ragusa

## Environment

In [1]:
import math
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import openmc
import openmc.mgxs

In [2]:
import os

# Add path to OpenMC binary
# os.environ['PATH'] += r':/path/to/openmc/bin'

# Add location of OpenMC xs data
# %env OPENMC_CROSS_SECTIONS=/path/to/endf/data/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml


## Initialize empty model

In [3]:
model = openmc.Model()

## Materials

In [4]:
uo2 = openmc.Material(name='UO2')
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_element('O', 2.0)
uo2.set_density('g/cm3', 10.0)

In [5]:
zirconium = openmc.Material(name='Zirc')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

In [6]:
water = openmc.Material(name='H2O')
water.add_element('H', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 0.7)

water.add_s_alpha_beta('c_H_in_H2O')

In [7]:
model.materials = openmc.Materials([uo2, zirconium, water])

## Geometry
### Surfaces

In [8]:
fuel_or = openmc.ZCylinder(r=0.39)
clad_ir = openmc.ZCylinder(r=0.40)
clad_or = openmc.ZCylinder(r=0.46)

pitch  = 1.26
left   = openmc.XPlane(x0 =-pitch/2, boundary_type='reflective')
right  = openmc.XPlane(x0 = pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(y0 =-pitch/2, boundary_type='reflective')
top    = openmc.YPlane(y0 = pitch/2, boundary_type='reflective')

### Regions based on surfaces

In [9]:
fuel_region  = -fuel_or
gap_region   = +fuel_or & -clad_ir
clad_region  = +clad_ir & -clad_or
water_region = +left & -right & +bottom & -top & +clad_or

### Cells (= regions + materials)

In [10]:
fuel_cell = openmc.Cell(name='Fuel')
fuel_cell.fill = uo2
fuel_cell.region = fuel_region

gap_cell = openmc.Cell(name='Gap')
# gap.fill = 'void'
gap_cell.region = gap_region

clad_cell = openmc.Cell(name='Clad')
clad_cell.fill = zirconium
clad_cell.region = clad_region

moderator_cell = openmc.Cell(name='Moderator')
moderator_cell.fill = water
moderator_cell.region = water_region

In [11]:
print(fuel_cell)
print(gap_cell)
print(clad_cell)
print(moderator_cell)

Cell
	ID             =	1
	Name           =	Fuel
	Fill           =	Material 1
	Region         =	-1
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None

Cell
	ID             =	2
	Name           =	Gap
	Fill           =	None
	Region         =	(1 -2)
	Rotation       =	None
	Translation    =	None
	Volume         =	None

Cell
	ID             =	3
	Name           =	Clad
	Fill           =	Material 2
	Region         =	(2 -3)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None

Cell
	ID             =	4
	Name           =	Moderator
	Fill           =	Material 3
	Region         =	(4 -5 6 -7 3)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None



## Universes

In [12]:
root_universe = openmc.Universe(cells=(fuel_cell, gap_cell, \
                                       clad_cell, moderator_cell))

In [13]:
model.geometry = openmc.Geometry(root_universe)

## Simulations parameters
### Source

In [14]:
# Create an initial uniform spatial source distribution over fissionable zones
bounds = [-pitch/2, -pitch/2, -pitch/2, pitch/2, pitch/2, pitch/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
source = openmc.IndependentSource(space=uniform_dist)

### particle/batches/basic tally

In [15]:
batches = 50
inactive = 10
particles = 250000

# Instantiate a Settings object
settings = openmc.Settings()

settings.source = source
settings.batches = batches
settings.inactive = inactive
settings.particles = particles
settings.run_mode = 'eigenvalue'

my_path = './mgxs_pin'
settings.output = {'tallies': False, 'path':my_path}

model.settings = settings

### check if folder exists

In [16]:
import pathlib
path = pathlib.Path(my_path)
path.mkdir(parents=True, exist_ok=True)

### pick energy-group structure

In [17]:
groups = openmc.mgxs.EnergyGroups(group_edges='XMAS-172')

## Tallies for MGXS generation
### Supported cross section types
MGXS_TYPES = (
    'total',
    'transport',
    'nu-transport',
    'absorption',
    'reduced absorption',
    'capture',
    'fission',
    'nu-fission',
    'kappa-fission',
    'scatter',
    'nu-scatter',
    'scatter matrix',
    'nu-scatter matrix',
    'multiplicity matrix',
    'nu-fission matrix',
    'scatter probability matrix',
    'consistent scatter matrix',
    'consistent nu-scatter matrix',
    'chi',
    'chi-prompt',
    'inverse-velocity',
    'prompt-nu-fission',
    'prompt-nu-fission matrix',
    'current',
    'diffusion-coefficient',
    'nu-diffusion-coefficient'
)

In [18]:
# Extract all Cells filled by Materials
openmc_cells = model.geometry.get_all_material_cells().values()

In [19]:
# Initialize MGXS Library
mgxs_lib = openmc.mgxs.Library(model.geometry)

# Add the chosen group structure
mgxs_lib.energy_groups = groups

# Scatttering Format and Legendre Order
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3

# Specify multi-group cross-section types to compute
mgxs_lib.mgxs_types = ['total', 'absorption', 'reduced absorption', 'scatter matrix', 'nu-scatter matrix',\
                       'consistent nu-scatter matrix', 'multiplicity matrix', \
                      'fission', 'nu-fission', 'kappa-fission','nu-fission matrix', 'chi', ]

# set uncertainty goal
mgxs_lib.tally_trigger = openmc.Trigger('std_dev', 1e-4)

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'cell'

# Specify the cell domains over which to compute multi-group cross sections
mgxs_lib.domains = model.geometry.get_all_material_cells().values()

# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies


/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


## Export model to XML

In [20]:
 model.export_to_model_xml(path = my_path + '/pincell.xml')

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=147.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=9.
  warn(msg, IDWarning)


## Run OpenMC

In [21]:
# trick to make several runs work with jupyter
try:
    sp 
    print('sp found')
    sp.close()
except NameError:
    print('sp NOT found')

sp NOT found


In [22]:
sp_file = model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

## Tally data processing

In [23]:
# Load the last statepoint file
sp = openmc.StatePoint(sp_file)

In [24]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [25]:
mgxs_file = mgxs_lib.create_mg_library(xs_type = 'macro')
mgxs_file.export_to_hdf5(filename = my_path + '/mgxs_pincell.h5')

In [26]:
sp.close()